# ROC CURVE and OTHER ANALYSIS

In [1]:
from deepface import DeepFace_last
import os
import numpy as np
import cv2;
import time
from sklearn.metrics import confusion_matrix, auc, roc_curve
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
import datetime
from collections import Counter
import pickle
from keras.models import load_model
import keras.losses
import tensorflow_addons as tfa
import pandas as pd
from keras.layers import Lambda

In [2]:
h5_model = "7_Facenet_OptSGD0.1_Batch512_Epoch50_prefetch15_Semihard_1stTrain"

In [3]:
with open("flod5_Validation_balancedFullDataset.pickle", "rb") as f:
    biggestList = pickle.load(f)
    


In [6]:
keras.losses.custom_loss = tfa.losses.TripletSemiHardLoss(margin=0.2)
model = load_model(f"performance_results\\Third_Repetition\\{h5_model}.h5")


In [7]:

start_time = time.time()

problem = []
response = []
for i in range(len(biggestList)):
    path1 = "E:\\Thesis\\EnhancedData\\flod_5\\Validation\\"+ biggestList[i][0]
    path2 = "E:\\Thesis\\EnhancedData\\flod_5\\Validation\\"+ biggestList[i][1]
    if i%1000 == 0:
        print(biggestList[i])
    response.append(DeepFace_last.verify(img1_path=path1, img2_path=path2,model=model ,normalization='Facenet', detector_backend="skip"))
    

['p146_bio.jpg', 'p146_1_0_0_glass.jpg']
['p726_out_bio.jpg', 'p722_out_0_3.jpg']


In [18]:
start_time = time.time()
all_results_of_all = dict()


res = f"performance_results\\Response_Validation_{h5_model}_TestedOn_Flod5.pickle"
with open(res, "rb") as f:
    response = pickle.load(f)
        
#---------------------------------------------------------------------------------------#

list_of_all_cosine = []
list_of_all_euclidean = []
list_of_all_euclidean_l2 = []

person = []

for tup in response:
    person.append(tup["img1_path"])
    list_of_all_cosine.append([tup["img1_path"], 
                               tup["img2_path"], 
                               tup["cosine_distance"]])

    list_of_all_euclidean.append([tup["img1_path"], 
                                  tup["img2_path"], 
                                  tup["euclidean_distance"]])

    list_of_all_euclidean_l2.append([tup["img1_path"], 
                                     tup["img2_path"],  
                                     tup["euclidean_l2_distance"]])
person = list(set(person))



#---------------------------------------------------------------------------------------#

thresholds_cosine = [i/600 for i in range(1000)] 
thresholds_euclidean = [i/100 for i in range(2,2500,15)]           
thresholds_euclidean_l2 = [i/100 for i in range(900)]

#---------------------------------------------------------------------------------------#

# Appending true labels

for i in range(len(list_of_all_cosine)):

# ---------

    bioIndex_cosine = list_of_all_cosine[i][0]
    selfieIndex_cosine = list_of_all_cosine[i][1]

    bioIndex_euclidean = list_of_all_euclidean[i][0]
    selfieIndex_euclidean = list_of_all_euclidean[i][1]

    bioIndex_euclidean_l2 = list_of_all_euclidean_l2[i][0]
    selfieIndex_euclidean_l2 = list_of_all_euclidean_l2[i][1]

# ---------
    if bioIndex_cosine.split("_")[0] == selfieIndex_cosine.split("_")[0]:

        if bioIndex_cosine.split("_")[1] == selfieIndex_cosine.split("_")[1] == "out":
            list_of_all_cosine[i].append(1)

        elif "out" in bioIndex_cosine or "out" in selfieIndex_cosine:
            list_of_all_cosine[i].append(0)

        else:
            list_of_all_cosine[i].append(1)

    else:
        list_of_all_cosine[i].append(0)


# ---------

    if bioIndex_euclidean.split("_")[0] == selfieIndex_euclidean.split("_")[0]:

        if bioIndex_euclidean.split("_")[1] == selfieIndex_euclidean.split("_")[1] == "out":
            list_of_all_euclidean[i].append(1)

        elif "out" in bioIndex_euclidean or "out" in selfieIndex_euclidean:
            list_of_all_euclidean[i].append(0)

        else:
            list_of_all_euclidean[i].append(1)

    else:
        list_of_all_euclidean[i].append(0)

# ---------

    if bioIndex_euclidean_l2.split("_")[0] == selfieIndex_euclidean_l2.split("_")[0]:

        if bioIndex_euclidean_l2.split("_")[1] == selfieIndex_euclidean_l2.split("_")[1] == "out":
            list_of_all_euclidean_l2[i].append(1)

        elif "out" in bioIndex_euclidean_l2 or "out" in selfieIndex_euclidean_l2:
            list_of_all_euclidean_l2[i].append(0)

        else:
            list_of_all_euclidean_l2[i].append(1)

    else:
        list_of_all_euclidean_l2[i].append(0)


#---------------------------------------------------------------------------------------#

# testing on a variety of thresholds and appending the predicted label:

for i in range(len(thresholds_cosine)):

    threshold = thresholds_cosine[i]

    for j in range(len(list_of_all_cosine)):
        distance = float(list_of_all_cosine[j][2])

        if distance <= threshold:
            list_of_all_cosine[j].append(["threshold: " + str(threshold), 1])

        else:
            list_of_all_cosine[j].append(["threshold: " + str(threshold), 0])

# ------

# testing on a variety of thresholds and appending the predicted label:

for i in range(len(thresholds_euclidean)):

    threshold = thresholds_euclidean[i]

    for j in range(len(list_of_all_euclidean)):
        distance = float(list_of_all_euclidean[j][2])

        if distance <= threshold:
            list_of_all_euclidean[j].append(["threshold: " + str(threshold), 1])

        else:
            list_of_all_euclidean[j].append(["threshold: " + str(threshold), 0])

# -------

# testing on a variety of thresholds and appending the predicted label:

for i in range(len(thresholds_euclidean_l2)):

    threshold = thresholds_euclidean_l2[i]

    for j in range(len(list_of_all_euclidean_l2)):
        distance = float(list_of_all_euclidean_l2[j][2])

        if distance <= threshold:
            list_of_all_euclidean_l2[j].append(["threshold: " + str(threshold), 1])

        else:
            list_of_all_euclidean_l2[j].append(["threshold: " + str(threshold), 0])

#---------------------------------------------------------------------------------------#

# Collecting real labels in a list

realLabels_cosine = []

for i in range(len(list_of_all_cosine)):
    realLabels_cosine.append(list_of_all_cosine[i][3])  


# -----

# Collecting real labels in a list

realLabels_euclidean = []

for i in range(len(list_of_all_euclidean)):
    realLabels_euclidean.append(list_of_all_euclidean[i][3])  


# -----

# Collecting real labels in a list

realLabels_euclidean_l2 = []

for i in range(len(list_of_all_euclidean_l2)):
    realLabels_euclidean_l2.append(list_of_all_euclidean_l2[i][3])  


#---------------------------------------------------------------------------------------#

# LAST STEP FOR COSINE

index_of_threshold_minFN_when_zeroFP = None

tpr_list_cosine = []
fpr_list_cosine = []

minFN_when_zeroFP = 10000
threshold_minFN_when_zeroFP = None
y_pred_forEveryThreshold_minFN_when_zeroFP = None

# Preparing lists of tpr and fpr by every single threshold point

highest_accuracy_cosine = 0
confusion_cosine = 0
best_threshold_cosine = 0

for i in range(len(thresholds_cosine)):

    y_pred_forEveryThreshold = []

    for j in range(len(list_of_all_cosine)):
        y_pred_forEveryThreshold.append(list_of_all_cosine[j][i+4][1])  # predictions based on every threshold 
                                                                 # had already been appended into the list_of_all

    cm = confusion_matrix(realLabels_cosine, y_pred_forEveryThreshold, labels=[0,1])
    # print("cm: ",cm)

    TN = cm[0, 0]
    # print("TN: ",TN)

    FP = cm[0,1]
    # print("FP: ",FP)

    FN = cm[1, 0]
    # print("FN is: ", FN)


    if FP == 0:
        if FN < minFN_when_zeroFP:
            minFN_when_zeroFP = FN
            threshold_minFN_when_zeroFP = thresholds_cosine[i]
            # y_pred_forEveryThreshold_minFN_when_zeroFP = y_pred_forEveryThreshold
            index_of_threshold_minFN_when_zeroFP = i+4

    TP = cm[1, 1]
    # print("TP is: ", TP)

    if TP == 0:
        tpr = 0
    else:
        tpr = TP/(TP+FN)
    # print("tpr is: ", tpr)

    fpr = 1 - TN/(TN+FP)
    # print("fpr is: ", fpr)

    tpr_list_cosine.append(tpr)
    fpr_list_cosine.append(fpr)

    acc = (TP+TN) / (TP+TN+FP+FN)
    if acc > highest_accuracy_cosine:
        highest_accuracy_cosine = acc
        confusion_cosine = cm
        best_threshold_cosine = thresholds_cosine[i]


# -----

roc_auc_cosine = auc(fpr_list_cosine, tpr_list_cosine)

#---------------------------------------------------------------------------------------#

cos_threshold_minFN_when_zeroFP=list_of_all_cosine[0][index_of_threshold_minFN_when_zeroFP]

person_number_of_TP = []
person_number_of_FN = []

for tup in list_of_all_cosine:
    if tup[0].split("_")[0] == tup[1].split("_")[0]:
        if ("out" in tup[0] and "out" in tup[1]) or ("out" not in tup[0] and "out" not in tup[1]):
            if tup[index_of_threshold_minFN_when_zeroFP][1] == 1:
                person_number_of_TP.append(tup[0])
            elif tup[index_of_threshold_minFN_when_zeroFP][1] == 0:
                person_number_of_FN.append(tup[0])
person_number_of_TP = list(set(person_number_of_TP))
person_number_of_FN = list(set(person_number_of_FN))

cos_person_number_of_TP_minFN_when_zeroFP=len(person_number_of_TP)
cos_ratio_of_person_FN_minFN_when_zeroFP_1TP=round((1-(len(person_number_of_TP)/len(person))) * 100, 2)
cos_ratio_of_person_FN_minFN_when_zeroFP_allTP=round((len(person_number_of_FN)/len(person)) * 100, 2)

#---------------------------------------------------------------------------------------#

# LAST STEP FOR EUCLIDEAN

index_of_threshold_minFN_when_zeroFP = None

tpr_list_euclidean = []
fpr_list_euclidean = []


minFN_when_zeroFP = 10000
threshold_minFN_when_zeroFP = None
y_pred_forEveryThreshold_minFN_when_zeroFP = None


# Preparing lists of tpr and fpr by every single threshold point

highest_accuracy_euclidean = 0
confusion_euclidean = 0
best_threshold_euclidean = 0

for i in range(len(thresholds_euclidean)):

    y_pred_forEveryThreshold = []

    for j in range(len(list_of_all_euclidean)):
        y_pred_forEveryThreshold.append(list_of_all_euclidean[j][i+4][1])  # predictions based on every threshold 
                                                                 # had already been appended into the list_of_all

    cm = confusion_matrix(realLabels_euclidean, y_pred_forEveryThreshold, labels=[0,1])
    # print("cm: ",cm)

    TN = cm[0, 0]
    # print("TN: ",TN)

    FP = cm[0,1]
    # print("FP: ",FP)

    FN = cm[1, 0]
    # print("FN is: ", FN)


    if FP == 0:
        if FN < minFN_when_zeroFP:
            minFN_when_zeroFP = FN
            threshold_minFN_when_zeroFP = thresholds_euclidean[i]
            # y_pred_forEveryThreshold_minFN_when_zeroFP = y_pred_forEveryThreshold
            index_of_threshold_minFN_when_zeroFP = i+4


    TP = cm[1, 1]
    # print("TP is: ", TP)

    if TP == 0:
        tpr = 0
    else:
        tpr = TP/(TP+FN)
    # print("tpr is: ", tpr)

    fpr = 1 - TN/(TN+FP)
    # print("fpr is: ", fpr)

    tpr_list_euclidean.append(tpr)
    fpr_list_euclidean.append(fpr)

    acc = (TP+TN) / (TP+TN+FP+FN)
    if acc > highest_accuracy_euclidean:
        highest_accuracy_euclidean = acc
        confusion_euclidean = cm
        best_threshold_euclidean = thresholds_euclidean[i]


# -----

roc_auc_euclidean = auc(fpr_list_euclidean, tpr_list_euclidean)

#---------------------------------------------------------------------------------------#

euc_threshold_minFN_when_zeroFP=list_of_all_euclidean[0][index_of_threshold_minFN_when_zeroFP]

person_number_of_TP = []
person_number_of_FN = []

for tup in list_of_all_euclidean:
    if tup[0].split("_")[0] == tup[1].split("_")[0]:
        if ("out" in tup[0] and "out" in tup[1]) or ("out" not in tup[0] and "out" not in tup[1]):
            if tup[index_of_threshold_minFN_when_zeroFP][1] == 1:
                person_number_of_TP.append(tup[0])
            elif tup[index_of_threshold_minFN_when_zeroFP][1] == 0:
                person_number_of_FN.append(tup[0])
person_number_of_TP = list(set(person_number_of_TP))
person_number_of_FN = list(set(person_number_of_FN))

euc_person_number_of_TP_minFN_when_zeroFP=len(person_number_of_TP)
euc_ratio_of_person_FN_minFN_when_zeroFP_1TP=round((1-(len(person_number_of_TP)/len(person))) * 100, 2)
euc_ratio_of_person_FN_minFN_when_zeroFP_allTP=round((len(person_number_of_FN)/len(person)) * 100, 2)

#---------------------------------------------------------------------------------------#

# LAST STEP FOR EUCLIDEAN_L2

index_of_threshold_minFN_when_zeroFP = None

tpr_list_euclidean_l2 = []
fpr_list_euclidean_l2 = []


minFN_when_zeroFP = 10000
threshold_minFN_when_zeroFP = None
y_pred_forEveryThreshold_minFN_when_zeroFP = None


# Preparing lists of tpr and fpr by every single threshold point

highest_accuracy_euclidean_l2 = 0
confusion_euclidean_l2 = 0
best_threshold_euclidean_l2 = 0

for i in range(len(thresholds_euclidean_l2)):

    y_pred_forEveryThreshold = []

    for j in range(len(list_of_all_euclidean_l2)):
        y_pred_forEveryThreshold.append(list_of_all_euclidean_l2[j][i+4][1])  # predictions based on every threshold 
                                                                 # had already been appended into the list_of_all

    cm = confusion_matrix(realLabels_euclidean_l2, y_pred_forEveryThreshold, labels=[0,1])
    # print("cm: ",cm)

    TN = cm[0, 0]
    # print("TN: ",TN)

    FP = cm[0,1]
    # print("FP: ",FP)

    FN = cm[1, 0]
    # print("FN is: ", FN)

    if FP == 0:
        if FN < minFN_when_zeroFP:
            minFN_when_zeroFP = FN
            threshold_minFN_when_zeroFP = thresholds_euclidean_l2[i]
            # y_pred_forEveryThreshold_minFN_when_zeroFP = y_pred_forEveryThreshold
            index_of_threshold_minFN_when_zeroFP = i+4

    TP = cm[1, 1]
    # print("TP is: ", TP)

    if TP == 0:
        tpr = 0
    else:
        tpr = TP/(TP+FN)
    # print("tpr is: ", tpr)

    fpr = 1 - TN/(TN+FP)
    # print("fpr is: ", fpr)

    tpr_list_euclidean_l2.append(tpr)
    fpr_list_euclidean_l2.append(fpr)

    acc = (TP+TN) / (TP+TN+FP+FN)
    if acc > highest_accuracy_euclidean_l2:
        highest_accuracy_euclidean_l2 = acc
        confusion_euclidean_l2 = cm
        best_threshold_euclidean_l2 = thresholds_euclidean_l2[i]


# -----

roc_auc_euclidean_l2 = auc(fpr_list_euclidean_l2, tpr_list_euclidean_l2)

#---------------------------------------------------------------------------------------#

eucL2_threshold_minFN_when_zeroFP=list_of_all_euclidean_l2[0][index_of_threshold_minFN_when_zeroFP]

person_number_of_TP = []
person_number_of_FN = []

for tup in list_of_all_euclidean_l2:
    if tup[0].split("_")[0] == tup[1].split("_")[0]:
        if ("out" in tup[0] and "out" in tup[1]) or ("out" not in tup[0] and "out" not in tup[1]):
            if tup[index_of_threshold_minFN_when_zeroFP][1] == 1:
                person_number_of_TP.append(tup[0])
            elif tup[index_of_threshold_minFN_when_zeroFP][1] == 0:
                person_number_of_FN.append(tup[0])
person_number_of_TP = list(set(person_number_of_TP))
person_number_of_FN = list(set(person_number_of_FN))

eucL2_person_number_of_TP_minFN_when_zeroFP=len(person_number_of_TP)
eucL2_ratio_of_person_FN_minFN_when_zeroFP_1TP=round((1-(len(person_number_of_TP)/len(person))) * 100, 2)
eucL2_ratio_of_person_FN_minFN_when_zeroFP_allTP=round((len(person_number_of_FN)/len(person)) * 100, 2)

#---------------------------------------------------------------------------------------#

tpr_cos = round((confusion_cosine[0, 0]/(confusion_cosine[0, 0] + confusion_cosine[0, 1])) * 100, 2)
tpr_euc = round((confusion_euclidean[0, 0]/(confusion_euclidean[0, 0] + confusion_euclidean[0, 1])) * 100, 2)
tpr_eucL2 = round((confusion_euclidean_l2[0, 0]/(confusion_euclidean_l2[0, 0] + confusion_euclidean_l2[0, 1])) * 100, 2)

fpr_cos = round((confusion_cosine[0, 1]/(confusion_cosine[0, 1] + confusion_cosine[0, 0])) * 100, 2)
fpr_euc = round((confusion_euclidean[0, 1]/(confusion_euclidean[0, 1] + confusion_euclidean[0, 0])) * 100, 2)
fpr_eucL2 = round((confusion_euclidean_l2[0, 1]/(confusion_euclidean_l2[0, 1] + confusion_euclidean_l2[0, 0])) * 100, 2)

#---------------------------------------------------------------------------------------#

# THREE CURVE IN ONE GRAPH


plt.figure(figsize = (10, 10))

sns.lineplot(x = fpr_list_cosine, y = tpr_list_cosine, linestyle="dashed", alpha=0.5, label='AUC (cosine) = %0.4f' % roc_auc_cosine)
sns.lineplot(x = fpr_list_euclidean, y = tpr_list_euclidean, linestyle="dashed", alpha=0.5, label='AUC (euclidean) = %0.4f' % roc_auc_euclidean)
sns.lineplot(x = fpr_list_euclidean_l2, y = tpr_list_euclidean_l2, linestyle="dashed", alpha=0.3, label='AUC (euclidean_l2) = %0.4f' % roc_auc_euclidean_l2)
#sns.lineplot(x = [0, 1], y = [0, 1], color ='red')

plt.xlim(-0.05, 1.05)
plt.ylim(-0.05, 1.05)
plt.legend(loc='lower right')

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

# GENERAL DESCRIPTION
plt.text(0.1, 
        0.9,"TL General Performance Based on Facenet Model", 
        fontsize=11)

# COSINE
plt.text(0.1, 
         0.3, 
         f"Cosine\nhighest accuracy:\n{round(highest_accuracy_cosine,4)}\n\n"
         f"best threshold:\n{best_threshold_cosine}\n\nTPR:\n{tpr_cos}\n\n"
         f"FPR:\n{fpr_cos}\n\ncm:\n{confusion_cosine}", 
         fontsize=12)

plt.text(0.1, 
         0.1, 
         f"Case of min(FN)|FP=0\n{cos_threshold_minFN_when_zeroFP}\n"
         f"#(TP)Person: {cos_person_number_of_TP_minFN_when_zeroFP}\n"
         f"Ratio_1TP: {cos_ratio_of_person_FN_minFN_when_zeroFP_1TP}\n"
         f"Ratio_allTP: {cos_ratio_of_person_FN_minFN_when_zeroFP_allTP}", 
         fontsize=12)

# EUCLIDEAN
plt.text(0.4, 
         0.3, 
         f"Euclidean\nhighest accuracy:\n{round(highest_accuracy_euclidean,4)}\n\n"
         f"best threshold:\n{best_threshold_euclidean}\n\nTPR:\n{tpr_euc}\n\n"
         f"FPR:\n{fpr_euc}\n\ncm:\n{confusion_euclidean}", 
         fontsize=12)

plt.text(0.4, 
         0.1, 
         f"Case of min(FN)|FP=0\n{euc_threshold_minFN_when_zeroFP}\n"
         f"#(TP)Person: {euc_person_number_of_TP_minFN_when_zeroFP}\n"
         f"Ratio_1TP: {euc_ratio_of_person_FN_minFN_when_zeroFP_1TP}\n"
         f"Ratio_allTP: {euc_ratio_of_person_FN_minFN_when_zeroFP_allTP}", 
         fontsize=12)

# EUCLIDEAN L2
plt.text(0.7, 
         0.3, 
         f"Euclidean_l2\nhighest accuracy:\n{round(highest_accuracy_euclidean_l2,4)}\n\n"
         f"best threshold:\n{best_threshold_euclidean_l2}\n\nTPR:\n{tpr_eucL2}\n\n"
         f"FPR:\n{fpr_eucL2}\n\ncm:\n{confusion_euclidean_l2}", 
         fontsize=12)

plt.text(0.7, 
         0.1, 
         f"Case of min(FN)|FP=0\n{eucL2_threshold_minFN_when_zeroFP}\n"
         f"#(TP)Person: {eucL2_person_number_of_TP_minFN_when_zeroFP}\n"
         f"Ratio_1TP: {eucL2_ratio_of_person_FN_minFN_when_zeroFP_1TP}\n"
         f"Ratio_allTP: {eucL2_ratio_of_person_FN_minFN_when_zeroFP_allTP}", 
         fontsize=12)

path_to_save_roc = r"performance_results"
# CHANGE THE PATH ABOVE
plt.savefig(path_to_save_roc+ f"\\ROC_Validation_{h5_model}_TestedOn_Flod5.jpg")
#plt.savefig(path_to_save_roc+ f"\\1_original_results.jpg")
plt.close()

#---------------------------------------------------------------------------------------#

index_in_all_results_dict = res


all_results_of_all[index_in_all_results_dict] = ["cosHighAcc",
                                                  round(highest_accuracy_cosine, 4), 
                                                 "cosBestThres",
                                                  best_threshold_cosine, 
                                                 "roc_auc_cosine",
                                                  roc_auc_cosine,
                                                 "tpr_cos",
                                                  tpr_cos, 
                                                 "fpr_cos",
                                                  fpr_cos, 
                                                 "cos_ratio_person_FN_1TP",
                                                  cos_ratio_of_person_FN_minFN_when_zeroFP_1TP,
                                                 "cos_ratio_person_FN_allTP",
                                                  cos_ratio_of_person_FN_minFN_when_zeroFP_allTP,
                                                 "eucHighAcc",
                                                  round(highest_accuracy_euclidean, 4), 
                                                 "eucBestThres",
                                                  best_threshold_euclidean, 
                                                 "roc_auc_euclidean",
                                                  roc_auc_euclidean,
                                                 "tpr_euc",
                                                  tpr_euc,
                                                 "fpr_euc",
                                                  fpr_euc, 
                                                 "euc_ratio_person_FN_1TP",
                                                  euc_ratio_of_person_FN_minFN_when_zeroFP_1TP,
                                                 "euc_ratio_person_FN_allTP",
                                                  euc_ratio_of_person_FN_minFN_when_zeroFP_allTP,
                                                 "eucL2HighAcc",
                                                  round(highest_accuracy_euclidean_l2, 4),
                                                 "eucL2BestThres",
                                                  best_threshold_euclidean_l2, 
                                                 "roc_auc_euclidean_l2",
                                                  roc_auc_euclidean_l2,
                                                 "tpr_eucL2",
                                                  tpr_eucL2,
                                                 "fpr_eucL2",
                                                  fpr_eucL2,
                                                 "eucL2_ratio_person_FN_1TP",
                                                  eucL2_ratio_of_person_FN_minFN_when_zeroFP_1TP,
                                                 "eucL2_ratio_person_FN_allTP",
                                                  eucL2_ratio_of_person_FN_minFN_when_zeroFP_allTP]

In [19]:
df_data = []
Euclidean = []
Euclidean.append(f'{roc_auc_euclidean:.2%}')
Euclidean.append(f'{highest_accuracy_euclidean:.2%}')
Euclidean.append(best_threshold_euclidean)
Euclidean.append(confusion_euclidean)
Euclidean.append(f'{tpr_euc}')
Euclidean.append(f'{fpr_euc}')

Euclidean_l2 = []
Euclidean_l2.append(f'{roc_auc_euclidean_l2:.2%}')
Euclidean_l2.append(f'{highest_accuracy_euclidean_l2:.2%}')
Euclidean_l2.append(best_threshold_euclidean_l2)
Euclidean_l2.append(confusion_euclidean_l2)
Euclidean_l2.append(f'{tpr_eucL2}')
Euclidean_l2.append(f'{fpr_eucL2}')

Cosine = []
Cosine.append(f'{roc_auc_cosine:.2%}')
Cosine.append(f'{highest_accuracy_cosine:.2%}')
Cosine.append(best_threshold_cosine)
Cosine.append(confusion_cosine)
Cosine.append(f'{tpr_cos}')
Cosine.append(f'{fpr_cos}')

In [20]:
df_data.append(Euclidean_l2)
df_data.append(Cosine)
df_data.append(Euclidean)

In [21]:
df = pd.DataFrame([Cosine,Euclidean_l2,Euclidean], index= ["Cosine","Euclidean_l2","Euclidean"], columns=["AUC","Accuracy","Threshold","Cm = tn, fp, fn, tp","TPR","FPR"])

In [22]:
df

,AUC,Accuracy,Threshold,"Cm = tn, fp, fn, tp",TPR,FPR
Cosine,99.90%,98.74%,0.695,"[[698, 15], [3, 710]]",97.9,2.1
Euclidean_l2,99.90%,98.74%,1.180,"[[698, 15], [3, 710]]",97.9,2.1
Euclidean,99.77%,97.76%,13.970,"[[690, 23], [9, 704]]",96.77,3.23


In [23]:
path_result = path_to_save_roc+ f"\\Results_Validation_{h5_model}_TestedOn_Flod5.pickle"
with open(path_result, "wb") as f:
    pickle.dump(all_results_of_all, f)

In [24]:
end_time = time.time()
print("Total time for the whole process", end_time - start_time)

Total time for the whole process 10.910949230194092
